# Testing the code from Chapter3 in the book

# C R U D operations

In [4]:
# Reduce logging
spark.sparkContext.setLogLevel("ERROR")

In [6]:
# Create database for these examples
spark.sql(
    """
        CREATE DATABASE IF NOT EXISTS exampleDB
    """
)

DataFrame[]

In [10]:
# Create test data

data = [
    (1, "United Kingdom", "London"),
    (2, "Canada", "Toronto")
]

schema = ["id", "country", "capital"]

df = spark.createDataFrame(data, schema=schema)

# Save as CSV, using pandas write in order to create single file
# PySpark separates the write and created additional files
# path = "/e/Programming/python_project/delta_lake_pyspark_learning/delta-lake-definitive-guide/datasets/my_datasets"
df.toPandas().to_csv("countries.csv", header=None)

# Save table as parquet for further usage
df.write.format("parquet").mode("overwrite").save("spark-warehouse/countries.parquet")

In [ ]:
!pwd

# C - Create

In [12]:
# Ensure table does not exist?!

spark.sql(
    """
DROP TABLE IF EXISTS exampleDB.countries
    """
)

DataFrame[]

In [13]:
spark.sql(
    """
CREATE TABLE exampleDB.countries (
    id LONG
    , country STRING
    , capital STRING
) USING DELTA
    """
)

DataFrame[]

How to Create a table in the working directory so it is inspectable


```
spark.sql("""
CREATE TABLE exampleDB.countries_loc (
  id LONG,
  country STRING,
  capital STRING
) USING DELTA
LOCATION '/opt/spark/work-dir/countries.delta';
""")
```

In [14]:
# Standard INSERT Operation

insert_sql = """
INSERT INTO exampleDB.countries VALUES
(1, "United Kingdom", "London")
, (2, "Canada", "Toronto")
"""

spark.sql(
    insert_sql
)

DataFrame[]

In [16]:
# INSERT INTO from parques file

spark.sql (
    """
    INSERT INTO exampleDB.countries
    SELECT * FROM parquet.`spark-warehouse/countries.parquet`
    """
)

DataFrame[]

In [17]:
# Simple CTAS statement
spark.sql(
"""
CREATE TABLE exampleDB.countries2 AS
SELECT * FROM exampleDB.countries
""" 
)

# Some issue with the chosen catalog, current one is derby which is in-memory temp catalog

DataFrame[]

In [19]:
# CONVERT TO DELTA
spark.sql(
    """
CONVERT TO DELTA parquet.`spark-warehouse/countries.parquet`
    """
)

DataFrame[]

#### Check out the _delta_log and view the contents of the JSON file(s)

In [ ]:
!tree spark-warehouse/countries.parquet/_delta_log

In [5]:
spark.sql(
    """
select * from exampleDB.countries
    """
).show(truncate=False, vertical=True)

-RECORD 0-----------------
 id      | 1              
 country | United Kingdom 
 capital | London         
-RECORD 1-----------------
 id      | 1              
 country | United Kingdom 
 capital | London         
-RECORD 2-----------------
 id      | 4              
 country | Romania        
 capital | Bucharest      
-RECORD 3-----------------
 id      | 6              
 country | Serbia         
 capital | Belgrade       
-RECORD 4-----------------
 id      | 2              
 country | Canada         
 capital | Toronto        
-RECORD 5-----------------
 id      | 2              
 country | Canada         
 capital | Toronto        
-RECORD 6-----------------
 id      | 3              
 country | Bulgaria       
 capital | Sofia          
-RECORD 7-----------------
 id      | 5              
 country | Grece          
 capital | Athens         



In [22]:
spark.table("exampleDB.countries").show()

+---+--------------+-------+
| id|       country|capital|
+---+--------------+-------+
|  1|United Kingdom| London|
|  1|United Kingdom| London|
|  2|        Canada|Toronto|
|  2|        Canada|Toronto|
+---+--------------+-------+



In [23]:
data2 = [
    (3, "Bulgaria", "Sofia")
    ,(4, "Romania", "Bucharest")
]

schema2 = ["id", "country", "capital"]

df2 = spark.createDataFrame(data2, schema2)

df2.write.format("delta").insertInto("exampleDB.countries")

In [24]:
data2 = [
    (5, "Grece", "Athens")
    ,(6, "Serbia", "Belgrade")
]

schema2 = ["id", "country", "capital"]

df2 = spark.createDataFrame(data2, schema2)

df2.write.format("delta").mode("append").saveAsTable("exampleDB.countries")

In [26]:
spark.table("exampleDB.countries").orderBy("id").show()

+---+--------------+---------+
| id|       country|  capital|
+---+--------------+---------+
|  1|United Kingdom|   London|
|  1|United Kingdom|   London|
|  2|        Canada|  Toronto|
|  2|        Canada|  Toronto|
|  3|      Bulgaria|    Sofia|
|  4|       Romania|Bucharest|
|  5|         Grece|   Athens|
|  6|        Serbia| Belgrade|
+---+--------------+---------+



In [30]:
# spark.sql("delete from exampleDB.countries where")

spark.sql(
    """
select row_number() over(partition by country order by id) as rn
from exampleDB.countries
order by id
    """
).show()

+---+
| rn|
+---+
|  1|
|  2|
|  1|
|  2|
|  1|
|  1|
|  1|
|  1|
+---+



# Querying data from Delta Lake Table

In [13]:
# Pythonic way using DeltaTable

from delta.tables import DeltaTable

countries_delta = DeltaTable.forName(spark, "exampleDB.countries")
countries_df = countries_delta.toDF()

countries_df.filter(countries_df.capital == "London").select("id", "country", "capital").show()

+---+-------+-------+
| id|country|capital|
+---+-------+-------+
+---+-------+-------+



 ## Read delta tables using Time Travel
 Time Travel allow us to query older versions of delta tables.  
 For instance: Create 1 table, then insert 2 records, then update 1 of them, then insert 1 new.  
 We have 4 versions so far, 1st empty table structure, 2nd with 2 records, 3rd 2 records 1 is updated, 4th 4 records.  

### VERSION AS OF

In [31]:
# SQL way
spark.sql(
    """
SELECT DISTINCT id 
FROM exampleDB.countries 
VERSION AS OF 2
ORDER BY ID ASC
    """
).show()

+---+
| id|
+---+
|  1|
|  2|
+---+



In [34]:
# Python way
# Paths in this cell must start with file: + windows path to directory
# Maybe because of the spark engine load logic OR simply because of the derby metastore? IDK
spark.read \
.option("versionAsOf", "1") \
.load("<your-path-to-table-here>") \
.select("id") \
.distinct() \
.orderBy("id") \
.show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
+---+



### DESRIBE HISTORY & TIMESTAMP AS OF

In case we use TIMESTAMP AS OF, we need accurate timestamp in order to retrieve any data.   
For that purpose we first use `DESCSRIBE HISTORY {DB.TABLE}` statement, to display the versions and take accurated timestamp.  
Then it is used in the SELECT SQL with time travel and response is returned.  

In [25]:
spark.sql(
    """
DESCRIBE HISTORY exampleDB.countries;
    """
).show(truncate=False, vertical=True)

-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 9                                                                                                                                                                                                                                                                                                                          
 timestamp           | 2025-02-24 19:31:03.42                                                                                                                                                                                                                                                                                     

In [16]:
# SQL way
spark.sql(
    """
    SELECT COUNT(*)
    FROM exampleDB.countries
    TIMESTAMP AS OF "2025-02-24 19:13:54"
    """
).show()

+--------+
|count(1)|
+--------+
|       6|
+--------+



In [42]:
# Python way
spark.read \
.option("timestampAsOf", "2025-02-24 19:12:42.331") \
.load("<your-path-to>/spark-warehouse/exampledb.db/countries") \
.count()


8

### Update
In this section, we are going to update a single records using SQL and Python way  

First update will change capital to 1 country with wrong city, then we will revert the action with another update

In [43]:
# SQL way
spark.sql(
"""
UPDATE exampleDB.countries
SET capital = "Ruse"
WHERE id = 3
""" 
)

DataFrame[num_affected_rows: bigint]

In [44]:
# Read table to see update
spark.sql(
    """
    SELECT *
    FROM exampleDB.countries
    WHERE id = 3
    """
).show()

+---+--------+-------+
| id| country|capital|
+---+--------+-------+
|  3|Bulgaria|   Ruse|
+---+--------+-------+



In [45]:
# Notes:
# - the new value in set clause must be wrapped first in double then in single quotes 
# Reason: Engine treats the value in double quotes as a column, if the single quotes are missing!

from pyspark.sql import functions as f

countries_delta.update(
    condition = f.expr("id = 3"),
    set = {"capital": "'Sofia'"}
)

In [4]:
# Read table to see update
spark.sql(
    """
    SELECT *
    FROM exampleDB.countries
    WHERE id = 3
    """
).show()

+---+-------+--------------+---+
| id|country|       capital|id2|
+---+-------+--------------+---+
|  3|    U.S|Washington D.C|  0|
+---+-------+--------------+---+



### Delete
In this section, we are going to DELETE records using SQL and Python way  


In [11]:
# SQL

spark.sql(
    """
DELETE FROM exampleDB.countries
WHERE id = 1
    """
)

DataFrame[num_affected_rows: bigint]

In [18]:
countries_df.select("id", "country", "capital").orderBy("id").show()

+---+--------+---------+
| id| country|  capital|
+---+--------+---------+
|  2|  Canada|  Toronto|
|  2|  Canada|  Toronto|
|  3|Bulgaria|    Sofia|
|  4| Romania|Bucharest|
|  5|   Grece|   Athens|
|  6|  Serbia| Belgrade|
+---+--------+---------+



In [19]:
# Pythonic way

countries_delta.delete("id = 2")

In [27]:
countries_df.select("id", "country", "capital").orderBy("id").show()

+---+--------+--------+
| id| country| capital|
+---+--------+--------+
|  3|Bulgaria|   Sofia|
|  5|   Grece|  Athens|
|  6|  Serbia|Belgrade|
+---+--------+--------+



In [23]:
from pyspark.sql import functions as f

countries_delta.delete(f.col("id") == 4)

In [26]:
countries_df.select("id", "country", "capital").orderBy("id").show()

+---+--------+--------+
| id| country| capital|
+---+--------+--------+
|  3|Bulgaria|   Sofia|
|  5|   Grece|  Athens|
|  6|  Serbia|Belgrade|
+---+--------+--------+



### Overwriting Data in a Delta Lake Table

In [6]:
# Python
new_data = [
    (1, "United Kingdom", "London", 1)
    ,(2, "Canada", "Toronto", 2)
    ,(3, "Bulgaria", "Sofia", 3)
    ,(4, "Romania", "Bucharest", 4)
    ,(5, "Grece", "Athens", 5)
    ,(6, "Serbia", "Belgrade", 6)
]

schema = ["id", "country", "capital", "id2"]

replace_df = spark.createDataFrame(data=new_data, schema=schema)

# Replace the schema with new
# Old records are removed, table is empty
delta_table2 = (
 DeltaTable.replace(spark)
 .tableName("exampleDB.countries")
 .addColumns(replace_df.schema)
 .execute()
 )



In [7]:
spark.sql(
    """
select * 
from exampleDB.countries 
order by id
    """
).show()

+---+-------+-------+---+
| id|country|capital|id2|
+---+-------+-------+---+
+---+-------+-------+---+



In [9]:
# Overwrite SPARK API
replace_df.write.format("delta").mode("overwrite").saveAsTable("exampleDB.countries")

In [22]:
spark.sql(
    """
select * 
from exampleDB.countries 
order by id
    """
).show()

+---+--------------+---------+
| id|       country|  capital|
+---+--------------+---------+
|  1|United Kingdom|   London|
|  2|        Canada|  Toronto|
|  3|      Bulgaria|    Sofia|
|  4|       Romania|Bucharest|
|  5|         Grece|   Athens|
|  6|        Serbia| Belgrade|
+---+--------------+---------+



In [40]:
# INSERT OVERWRITE - Overwrites entire table, with the data from VALUES

spark.sql(
    """
INSERT OVERWRITE exampleDB.countries
VALUES(3, 'U.S', "Washington D.C", 0)
    """
)

DataFrame[]

In [5]:
spark.sql(
    """
select * 
from exampleDB.countries 
order by id
    """
).show()

+---+-------+--------------+---+
| id|country|       capital|id2|
+---+-------+--------------+---+
|  3|    U.S|Washington D.C|  0|
+---+-------+--------------+---+



In [21]:
spark.sql(
    """
DESCRIBE HISTORY exampleDB.countries;
    """
).show(truncate=False, vertical=True)

-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 17                                                                                                                                                                                                                                                                                                                         
 timestamp           | 2025-03-01 19:21:56.319                                                                                                                                                                                                                                                                                    

In [19]:
# In order to drop a column in next cell
# I had to update the table properties to the ones in SQL below

spark.sql(
    """
        ALTER TABLE exampleDB.countries 
        SET TBLPROPERTIES (
            'delta.columnMapping.mode' = 'name',
            'delta.minReaderVersion' = '2',
            'delta.minWriterVersion' = '5'
        )
    """)

DataFrame[]

In [20]:
# See comment in cell above
spark.sql("ALTER TABLE exampleDB.countries DROP COLUMN id2")

DataFrame[]

In [37]:
spark.sql("select * from exampleDB.countries2").show()

+---+--------------------+---------+
| id|             country|  capital|
+---+--------------------+---------+
|  7|              France|    Paris|
|  8|             Germany|   Berlin|
|  9|Dutchland/Netherl...|Amsterdam|
| 10|               Italy|     Rome|
+---+--------------------+---------+



In [34]:
# Content in cell above was different before truncating, it had duplicated
spark.sql('TRUNCATE TABLE exampleDB.countries2')

DataFrame[]

### MERGE / UPSERT

In [36]:
spark.sql(
    """
INSERT INTO exampleDB.countries2
VALUES (7, 'France', 'Paris')
, (8, 'Germany', 'Berlin')
, (9, 'Dutchland/Netherlands', 'Amsterdam')
, (10, 'Italy', 'Rome')
    """
)

DataFrame[]

In [40]:
# Inserting the correct pairs of country/capital in order to use it as a source for the merge statement
spark.sql(
    """
INSERT INTO exampleDB.countries2
VALUES (11, 'Spain', "Madrid") -- Madrid
, (12, 'Belgium', 'Brussels') -- Brussels
, (13, 'Czech Republic', 'Prague') -- Prague
    """
)

DataFrame[]

In [39]:
# Inserting few rows with wrong capitals in order to update them with MERGE statement
spark.sql(
    """
INSERT INTO exampleDB.countries
VALUES (11, 'Spain', "Barcelona") -- Madrid
, (12, 'Belgium', 'Liège') -- Brussels
, (13, 'Czech Republic', 'Brno') -- Prague
    """
)

DataFrame[]

In [41]:
# SQL

spark.sql(
    """
        MERGE INTO exampleDB.countries A -- Target
        USING (SELECT * FROM exampleDB.countries2) B -- Source
            ON A.id = B.id -- Matching condition, mostly PK is used
        WHEN MATCHED THEN -- Action when A.PK = B.PK
          UPDATE SET -- Action
            id = A.id
            , country = B.country
            , capital = B.capital
        WHEN NOT MATCHED -- Action when PK not match
            THEN INSERT ( -- Action
                id
                , country
                , capital
            )
            VALUES (
                B.id
                , B.country
                , B.capital
            )
    """
)

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [43]:
spark.sql("select * from exampleDB.countries ORDER BY id").show()

+---+--------------------+---------+
| id|             country|  capital|
+---+--------------------+---------+
|  1|      United Kingdom|   London|
|  2|              Canada|  Toronto|
|  3|            Bulgaria|    Sofia|
|  4|             Romania|Bucharest|
|  5|               Grece|   Athens|
|  6|              Serbia| Belgrade|
|  7|              France|    Paris|
|  8|             Germany|   Berlin|
|  9|Dutchland/Netherl...|Amsterdam|
| 10|               Italy|     Rome|
| 11|               Spain|   Madrid|
| 12|             Belgium| Brussels|
| 13|      Czech Republic|   Prague|
+---+--------------------+---------+



In [44]:
# Preparation for Python merge
spark.sql(
    """
        INSERT INTO exampleDB.countries
        VALUES (14, 'Norway', "Bergen") -- Oslo is the correct one 
    """
)

DataFrame[]

In [45]:
# Preparation for new UPSERT source
source_df = (
    spark.createDataFrame(
        [
            (14, 'Norway', "Oslo")
            ,(15, 'Monaco', 'Monaco') 
        ], schema=['id', 'country', 'capital']
    )
)


In [47]:
# UPSERT with Delta API
from delta.tables import DeltaTable
# Create DeltaTable object
countries_delta = DeltaTable.forName(spark, "exampleDB.countries")

countries_delta.alias('target').merge(
    source = source_df.alias('source')
    , condition = 'source.id = target.id'
).whenMatchedUpdate(
    set = {
        'country': 'source.country'
        , 'capital': 'source.capital'
    }
).whenNotMatchedInsert(
    values = {
        'id': 'source.id'
        , 'country': 'source.country'
        , 'capital': 'source.capital'
    }
).execute()

In [46]:
# Executed before MERGE Statement above
spark.sql("select * from exampleDB.countries ORDER BY id").show()

+---+--------------------+---------+
| id|             country|  capital|
+---+--------------------+---------+
|  1|      United Kingdom|   London|
|  2|              Canada|  Toronto|
|  3|            Bulgaria|    Sofia|
|  4|             Romania|Bucharest|
|  5|               Grece|   Athens|
|  6|              Serbia| Belgrade|
|  7|              France|    Paris|
|  8|             Germany|   Berlin|
|  9|Dutchland/Netherl...|Amsterdam|
| 10|               Italy|     Rome|
| 11|               Spain|   Madrid|
| 12|             Belgium| Brussels|
| 13|      Czech Republic|   Prague|
| 14|              Norway|   Bergen|
+---+--------------------+---------+



In [48]:
# Executed after MERGE Statement above
spark.sql("select * from exampleDB.countries ORDER BY id").show()

+---+--------------------+---------+
| id|             country|  capital|
+---+--------------------+---------+
|  1|      United Kingdom|   London|
|  2|              Canada|  Toronto|
|  3|            Bulgaria|    Sofia|
|  4|             Romania|Bucharest|
|  5|               Grece|   Athens|
|  6|              Serbia| Belgrade|
|  7|              France|    Paris|
|  8|             Germany|   Berlin|
|  9|Dutchland/Netherl...|Amsterdam|
| 10|               Italy|     Rome|
| 11|               Spain|   Madrid|
| 12|             Belgium| Brussels|
| 13|      Czech Republic|   Prague|
| 14|              Norway|     Oslo|
| 15|              Monaco|   Monaco|
+---+--------------------+---------+



### Other Useful Actions
- Parquet Conversions
- Other Useful Actions

In [ ]:
# Parquet/Iceberg Conversion - Not executed, only written for practice
# Parquet conversion was already tested in the cells in above sections

In [ ]:
# SQL
# Important Note: If table is already partition, it is required to specify the partition columns with a PARTITION BY clause
spark.sql( 
    """
        CONVERT TO DELTA parquet`spark-warehouse/countries.parquet`
    """
)

In [52]:
# Delta API
from delta.tables import DeltaTable

converted_delta_table = (
    DeltaTable.convertToDelta(
        spark,
        "parquet.`spark-warehouse/countries.parquet`"
    )
)

In [ ]:
'''Iceberg conversion
The DeltaTable API does not support the Iceberg
conversion. Spark SQL, however, can support the conversion with CONVERT TO
DELTA, but you will also need to install support for the Iceberg format in your Spark
environment -> additional JAR file (delta-iceberg)
Unlike conversion from parquet, iceberg conversion infers PARTITIONING metadata from source

There’s one more thing you should know about this conversion process. An interest‐
ing side effect exists in converted Iceberg tables. Since both Iceberg and Delta Lake
maintain distinctly separate transaction logs, none of the new files added through
interactions via Delta Lake will be registered on the Iceberg side. However, since
the Iceberg log is not removed, the new Delta Lake table will still be readable and
accessible as an Iceberg table

SQL:
'''
spark.sql("CONVERT TO DELTA iceberg.`countries.iceberg`")

In [ ]:
# Delta Lake Metadata and History

In [ ]:
# SQL
spark.sql("DESCRIBE DETAIL exampleDB.countries2").show(truncate=False, vertical=True)

In [ ]:
# Python
# countries_delta.details() - Returns Spark DataFrame
countries_delta.detail().show(truncate=False, vertical=True)

In [60]:
# SQL
spark.sql("DESCRIBE HISTORY exampleDB.countries").show(truncate=False, vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 21                                                                                                                                                                                                                                                                                                                               

In [62]:
# Python
countries_delta.history().show(truncate=False, vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 21                                                                                                                                                                                                                                                                                                                               

# Finished!